In [2]:
%pylab inline

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

# numerical processing and scientific libraries
import numpy as np

# plotting
import matplotlib.pyplot as plt

# reading wav and mp3 files
from audiofile_read import *  # included in the rp_extract git package

# Rhythm Pattern Audio Extraction Library
from rp_extract import rp_extract
from rp_plot import *   # can be skipped if you don't want to do any plots
import librosa
import os
import pandas as pd
import scipy

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
samplerate, samplewidth, wavedata = audiofile_read('../audio/electro/01-max_cooper_tom_hodge-symmetry.mp3', normalize=False)

NameError: File does not exist: ../audio/electro/01-max_cooper_tom_hodge-symmetry.mp3

In [3]:
def features_extraction_accompaniment(track_path):
    # adapt the fext array to your needs:
    fext = ['rp','ssd','rh','mvd'] # sh, tssd, trh
    samplerate, samplewidth, wavedata = audiofile_read(track_path, normalize=False)
    features = rp_extract(wavedata,
                      samplerate,
                      extract_rp   = ('rp' in fext),          # extract Rhythm Patterns features
                      extract_ssd  = ('ssd' in fext),           # extract Statistical Spectrum Descriptor
                      #extract_sh   = ('sh' in fext),          # extract Statistical Histograms
                      extract_tssd = ('tssd' in fext),          # extract temporal Statistical Spectrum Descriptor
                      extract_rh   = ('rh' in fext),           # extract Rhythm Histogram features
                      extract_trh  = ('trh' in fext),          # extract temporal Rhythm Histogram features
                      extract_mvd  = ('mvd' in fext),        # extract Modulation Frequency Variance Descriptor
                      spectral_masking=True,
                      transform_db=True,
                      transform_phon=True,
                      transform_sone=True,
                      fluctuation_strength_weighting=True,
                      skip_leadin_fadeout=1,
                      step_width=1)
    res = []
    for key in fext:
        res.extend(features[key])
    return res

In [5]:
a = features_extraction_accompaniment('../audio/electro/01-max_cooper_tom_hodge-symmetry.mp3')

Decoded .mp3 with: ffmpeg -v 1 -y -i ../audio/electro/01-max_cooper_tom_hodge-symmetry.mp3 C:\Users\Admin\AppData\Local\Temp\b65a0ab4-44ce-4ea5-8758-9042cd5994de.wav


In [4]:
def features_extraction_voice(track_path):
#     mfccs = librosa.feature.mfcc(wavedata, sr=sr,n_mfcc=26)
#     deltas = librosa.feature.delta(mfccs)
    wavedata, sr = librosa.load(track_path)
    discretization = np.linspace(0, len(wavedata),4,dtype=int)
    mfccs = []
    deltas = []
    for i in range(len(discretization) - 1):
        mfccs_tmp = librosa.feature.mfcc(wavedata[discretization[i]:discretization[i+1]], sr=sr,n_mfcc=26)
        mfccs.extend(np.median(mfccs_tmp,axis=1)[:12])
        deltas.extend(np.median(mfccs_tmp,axis=1)[:12])
    return mfccs + deltas
    

In [7]:
b = features_extraction_voice('../audio/electro/01-max_cooper_tom_hodge-symmetry.mp3')

In [5]:
def track_preprocessing(track_path):
    return features_extraction_accompaniment(track_path) + features_extraction_voice(track_path)

In [9]:
c = track_preprocessing('../audio/electro/01-max_cooper_tom_hodge-symmetry.mp3')

Decoded .mp3 with: ffmpeg -v 1 -y -i ../audio/electro/01-max_cooper_tom_hodge-symmetry.mp3 C:\Users\Admin\AppData\Local\Temp\9093df8f-0926-4571-a1c2-9e2569158425.wav


In [51]:
# calculating distance between tracks
for i in os.listdir('../audio/')[1:]:
    for j in os.listdir('../audio/'+i+'/')[1:]:
        tmp = track_preprocessing('../audio/'+i+'/'+j)
        pd.DataFrame(tmp).to_csv('features_final/'+j[:-4]+'.csv',header=None,index=None)
        

Decoded .mp3 with: ffmpeg -v 1 -y -i ../audio/classics/01. Lesley Garrett - Caccini Ave Maria.mp3 C:\Users\Admin\AppData\Local\Temp\dccf4f2b-5587-4471-ac18-9b3e0175ffe3.wav
Decoded .mp3 with: ffmpeg -v 1 -y -i ../audio/classics/01. Russell Watson - Volare.mp3 C:\Users\Admin\AppData\Local\Temp\326aa360-e422-4e3b-82d2-ebf979955cdd.wav
Decoded .mp3 with: ffmpeg -v 1 -y -i ../audio/classics/02. Charlotte Church - Panis Angelicus.mp3 C:\Users\Admin\AppData\Local\Temp\eb1498af-efa0-40be-ba1d-c07ed77a3f3f.wav
Decoded .mp3 with: ffmpeg -v 1 -y -i ../audio/classics/02. Russell Watson - Mattinata.mp3 C:\Users\Admin\AppData\Local\Temp\4c920075-17d5-48c4-b232-b7e13843319c.wav
Decoded .mp3 with: ffmpeg -v 1 -y -i ../audio/classics/03. Angela Gheorghiu - Ebben ...Ne andro lontana (La Wally).mp3 C:\Users\Admin\AppData\Local\Temp\05e2d183-9bf1-4e56-bbf3-4a44742e6658.wav
Decoded .mp3 with: ffmpeg -v 1 -y -i ../audio/classics/03. Bond - Viva!.mp3 C:\Users\Admin\AppData\Local\Temp\5d0dfff0-1af2-45a7-9f1c

In [65]:
d = {}
for file in os.listdir('features_final/'):
    d[file] = pd.read_csv('features_final/'+file,header=None)[0].values

In [69]:
distances = pd.DataFrame(columns=d.keys(),index=d.keys())

In [73]:
for basic_track in d:
    for compared_track in d:
        distances.loc[basic_track,compared_track] = scipy.spatial.distance.cosine(d[basic_track],d[compared_track])

In [75]:
distances.to_excel('tracks_similarity_matrix.xlsx')

In [84]:
recommendations = {}
for col in distances.columns:
    recommendations[col] = distances[col].sort_values()[:20].index

In [88]:
pd.DataFrame(recommendations).to_excel('recommendations_for_tracks.xlsx',index=None)

In [6]:
def recommendation_for_new_track(tracks_path):
    tp = track_preprocessing(tracks_path)
    d = {}
    for file in os.listdir('features_final/'):
        d[file] = pd.read_csv('features_final/'+file,header=None)[0].values
    dist = pd.DataFrame(columns=['dist'],index=d.keys())
    for col in d.keys():
        dist.loc[col,'dist'] = scipy.spatial.distance.cosine(d[col],tp)
    return dist['dist'].sort_values()[:20].index

In [8]:
recommendation_for_new_track('../audio/electro/01-max_cooper_tom_hodge-symmetry.mp3')

Decoded .mp3 with: ffmpeg -v 1 -y -i ../audio/electro/01-max_cooper_tom_hodge-symmetry.mp3 C:\Users\Admin\AppData\Local\Temp\6b3f7031-e37a-4fa8-b896-971985f06235.wav


Index(['01-max_cooper_tom_hodge-symmetry.csv',
       '10. Crucify Me (Feat. Lights).csv', '05. Chelsea Smile.csv',
       '06. The Comedown.csv',
       '04. The Sadness Will Never End (Feat. Sam Carter).csv',
       '02. Pray For Plagues.csv', '03 - You Kill Me with Sirence.csv',
       '01. Traitors Never Play Hang-Man.csv', '11 - Only In Dreams.csv',
       '16 - Northern Lights.csv', '03. Diamonds Aren't Forever.csv',
       '08. It Never Ends.csv', '07. Blessed With A Curse.csv',
       '05 - Face For Today.csv', '09 - Change The Skyline.csv',
       '02 - Last Night In The City.csv', '06-max_cooper-order_from_chaos.csv',
       '10 - Butterfly Girl.csv',
       '04. Hans Zimmer - Now We Are Free (Gladiator).csv',
       '13 - Planet Roaring.csv'],
      dtype='object')